In [1]:
import os
import pandas as pd
from glob import glob

In [2]:
base_dir = os.path.join('C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/HAM_TEST_Project/data/KNU/' )
all_image_path = glob(os.path.join(base_dir, '*', '*.jpg'))
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}
imageid_list = imageid_path_dict.keys()

In [39]:
def get_data(base_dir, imageid_path_dict):
    # image_id_path_dict = {'SCC_9_3_1_head&neck':'C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/HAM_TEST_Project/data/KNU\\data\\SCC_9_3_1_head&neck.jpg', ...}
    label_type_dict = {
    'BCC': 1,
    'SCC': 2,
    'Melanoma': 3,
    'Benign': 0
    }
    site_type_dict = {
    'head&neck': 0,
    'extremity': 1,
    'trunk': 2,
    'acral': 3
    }
    df_original = pd.DataFrame(columns = ['orig_name','subject_no','image_no','lesion_no','path','site','site_label','cell_type','label','mode'])

    df_original['orig_name'] = imageid_path_dict.keys()
    df_original['path'] = df_original['orig_name'].map(imageid_path_dict.get)

    df_original['subject_no'] = df_original['orig_name'].apply(lambda x: int(x.split('_')[1]))
    df_original['image_no'] = df_original['orig_name'].apply(lambda x: int(x.split('_')[2]))
    df_original['lesion_no'] = df_original['orig_name'].apply(lambda x: int(x.split('_')[3]))
    df_original['site'] = df_original['orig_name'].apply(lambda x: x.split('_')[4])
    df_original['site_label'] = df_original['site'].map(site_type_dict.get)
    df_original['cell_type'] = df_original['orig_name'].apply(lambda x: x.split('_')[0])
    df_original['label'] = df_original['site'].map(label_type_dict.get)

    # Train-Test Split
    df_permutated = df_original.sample(frac=1,random_state=0)              # 섞기
    split = 0.8
    

    train_ind_list=[]
    test_ind_list =[]

    df_train = pd.DataFrame()
    df_test = pd.DataFrame()

    for label in label_type_dict.keys():
        df_label = df_original.loc[df_original['cell_type']==label]
        patients = list(df_label['subject_no'].unique())
        len_train = int(len(patients)*0.8)
        patients_train = patients[:len_train]
        patients_test = patients[len_train:]
        train = df_label.loc[df_label['subject_no'].isin(patients_train)]
        test = df_label.loc[df_label['subject_no'].isin(patients_test)]

        df_train = df_train.append(train)
        df_test = df_test.append(test)
        
    df_train['mode']='train'
    df_test['mode']='test'

    df_undup = pd.concat([df_train,df_test])
    return df_undup



In [40]:
a = get_data(base_dir, imageid_path_dict)

In [41]:
a

,orig_name,subject_no,image_no,lesion_no,path,site,site_label,cell_type,label,mode
0,BCC_0_1_1_head&neck,0,1,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,BCC,None,train
1,BCC_100_1_1_head&neck,100,1,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,BCC,None,train
2,BCC_100_2_1_head&neck,100,2,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,BCC,None,train
3,BCC_100_3_1_head&neck,100,3,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,BCC,None,train
4,BCC_101_1_1_head&neck,101,1,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,BCC,None,train
...,...,...,...,...,...,...,...,...,...,...
2797,Benign_99_3_1_head&neck,99,3,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,Benign,None,test
2798,Benign_99_3_2_head&neck,99,3,2,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,Benign,None,test
2799,Benign_9_1_1_head&neck,9,1,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,Benign,None,test
2800,Benign_9_2_1_head&neck,9,2,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,Benign,None,test


In [42]:
a.loc[a['mode']=='train'].shape

(3980, 10)

In [43]:
a.loc[a['mode']=='test'].shape

(1000, 10)

In [47]:
a.loc[a['subject_no']==6]

,orig_name,subject_no,image_no,lesion_no,path,site,site_label,cell_type,label,mode
1220,BCC_6_1_1_head&neck,6,1,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,BCC,None,test
1221,BCC_6_2_1_head&neck,6,2,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,BCC,None,test
4847,SCC_6_1_1_head&neck,6,1,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,SCC,None,test
4848,SCC_6_2_1_head&neck,6,2,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,SCC,None,test
4849,SCC_6_3_1_head&neck,6,3,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,SCC,None,test
2674,Benign_6_1_1_head&neck,6,1,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,Benign,None,test
2675,Benign_6_2_1_head&neck,6,2,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,Benign,None,test
2676,Benign_6_3_1_head&neck,6,3,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,Benign,None,test


In [6]:
b = a.sample(frac=1,random_state=0)
b

,orig_name,subject_no,image_no,lesion_no,path,site,site_label,cell_type,label,mode
2484,Benign_391_2_1_head&neck,391,2,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,Benign,None,NaN
3950,Melanoma_92_4_1_acral,92,4,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,acral,3,Melanoma,None,NaN
3956,Melanoma_93_4_1_head&neck,93,4,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,Melanoma,None,NaN
4878,SCC_77_3_1_head&neck,77,3,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,SCC,None,NaN
4155,SCC_134_1_1_head&neck,134,1,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,SCC,None,NaN
...,...,...,...,...,...,...,...,...,...,...
4931,SCC_89_6_1_acral,89,6,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,acral,3,SCC,None,NaN
3264,Melanoma_183_4_1_acral,183,4,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,acral,3,Melanoma,None,NaN
1653,Benign_176_3_1_acral,176,3,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,acral,3,Benign,None,NaN
2607,Benign_4_1_1_head&neck,4,1,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,Benign,None,NaN


In [8]:
tmp = []
tmp2 = list(b.index[0:3])

In [9]:
tmp +=tmp2

In [10]:
tmp

[2484, 3950, 3956]

In [17]:
a.iloc[b.index[0:3]]

,orig_name,subject_no,image_no,lesion_no,path,site,site_label,cell_type,label,mode
2484,Benign_391_2_1_head&neck,391,2,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,Benign,None,NaN
3950,Melanoma_92_4_1_acral,92,4,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,acral,3,Melanoma,None,NaN
3956,Melanoma_93_4_1_head&neck,93,4,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,Melanoma,None,NaN


In [108]:
label_type_dict = {
    'BCC': 1,
    'SCC': 2,
    'Melanoma': 3,
    'Benign': 0
    }
df_permutated = b.sample(frac=1,random_state=0)              # 섞기
split = 0.8


train_ind_list=[]
test_ind_list =[]

for label in label_type_dict.keys():
    df_label = b.loc[df_permutated['cell_type']==label]
    train_size = int(len(df_label)*split)
    train_ind = list(df_label.index[:train_size])
    test_ind = list(df_label.index[train_size:])
    train_ind_list += train_ind
    test_ind_list += test_ind

In [107]:
print(len(train_ind_list))
print(len(test_ind_list))

3983
997


In [112]:
df_label
pat = df_label['subject_no'].unique()

In [115]:
import numpy as np
pat_train = np.random.choice(pat,int(len(pat)*0.8))

In [117]:
int(len(pat)*0.8)

328

In [119]:
int(len(pat)*0.2)

82

In [120]:
len(pat)

411

In [3]:
subject_map = pd.read_csv('C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/KNU_Project/data/KNU_DSLR/KNU_DSLR_subjectmap.csv')

In [5]:
subject_map.drop_duplicates(subset=['subject_no','label']).reset_index()

,index,orig_name,subject_no,image_no,lesion_no,path,site,site_label,cell_type,label,mode
0,0,BCC_0_1_1_head&neck,0,1,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,BCC,1,train
1,1,BCC_100_1_1_head&neck,100,1,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,BCC,1,train
2,4,BCC_101_1_1_head&neck,101,1,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,BCC,1,train
3,7,BCC_102_1_1_head&neck,102,1,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,BCC,1,train
4,10,BCC_103_1_1_head&neck,103,1,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,BCC,1,train
...,...,...,...,...,...,...,...,...,...,...,...
1330,4951,Benign_96_1_1_trunk,96,1,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,trunk,2,Benign,0,test
1331,4954,Benign_97_1_1_head&neck,97,1,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,Benign,0,test
1332,4965,Benign_98_1_1_head&neck,98,1,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,Benign,0,test
1333,4971,Benign_99_1_1_head&neck,99,1,1,C:/Users/User/Desktop/김세연/SKINCANCER_DRANSWER/...,head&neck,0,Benign,0,test
